In [ ]:
pip install ktrain


In [ ]:
import ktrain

In [ ]:
# !pip install tensorflow==2.15.1
# import os
# # os.environ['TF_USE_LEGACY_KERAS'] = '1' # This is no longer needed

# import tensorflow as tf
# import ktrain

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
import ktrain
from ktrain import text

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/datasets/SMSSpamCollection', sep = '\t', names = ['label', 'text'])

In [ ]:
df

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...


In [ ]:
from sklearn.model_selection import train_test_split
data_train, data_test = train_test_split(df, test_size = 0.2, random_state = 0)

In [ ]:
data_train.shape, data_test.shape


((4457, 2), (1115, 2))

### data prepration

In [ ]:
(X_train, y_train), (X_test, y_test), preproc = text.texts_from_df(train_df = data_train,
                                                                   text_column = 'text',
                                                                   label_columns = 'label',
                                                                   val_df = data_test,
                                                                   preprocess_mode = 'bert') # or you can write ditilbert

['ham', 'spam']
      ham  spam
1114  1.0   0.0
3589  1.0   0.0
3095  1.0   0.0
1012  1.0   0.0
3320  1.0   0.0
['ham', 'spam']
      ham  spam
4456  1.0   0.0
690   0.0   1.0
944   1.0   0.0
3768  1.0   0.0
1189  1.0   0.0
downloading pretrained BERT model (uncased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: en


Is Multi-Label? False
preprocessing test...
language: en


In [15]:
y_train

array([[1., 0.],
       [1., 0.],
       [1., 0.],
       ...,
       [0., 1.],
       [1., 0.],
       [1., 0.]], dtype=float32)

In [16]:
y_test

array([[1., 0.],
       [0., 1.],
       [1., 0.],
       ...,
       [1., 0.],
       [0., 1.],
       [1., 0.]], dtype=float32)

In [20]:
X_train, '-----------------', len(X_train)

([array([[  101,  2053,  1045, ...,     0,     0,     0],
         [  101,  2065,  2017, ...,     0,     0,     0],
         [  101,  2031,  2017, ...,     0,     0,     0],
         ...,
         [  101,  2005, 24471, ...,     0,     0,     0],
         [  101,  1054,  1057, ...,     0,     0,     0],
         [  101,  3461,  3110, ...,     0,     0,     0]]),
  array([[0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         ...,
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0],
         [0, 0, 0, ..., 0, 0, 0]])],
 '-----------------',
 2)

 this code defines a text classification model based on either BERT or DistilBERT, feeds it training data, and uses a preprocessor to get the data ready for the model. The resulting model object can then be used for further training and prediction.

In [21]:
## use 'distilbert' if you want
model = text.text_classifier(name = 'bert', # or distilebert
                           train_data = (X_train, y_train),
                           preproc = preproc)

Is Multi-Label? False
maxlen is 400


/usr/local/lib/python3.10/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer GlorotNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


done.


In [22]:
## here we have taken batch size as 6 as from the documentation it is recommend to use this with maxlen as 500
learner = ktrain.get_learner(model = model, train_data = (X_train, y_train), val_data = (X_test, y_test), batch_size = 6) # preparing the model for training

In [23]:
learner

lr_find(): This is a method of the Learner object. It's designed to help you find a good learning rate for your model. A learning rate is a crucial hyperparameter that controls how much the model's weights are adjusted during training. Choosing a good learning rate can significantly impact the performance of your model.

In [ ]:
learner.lr_find(show_plot=True, max_epochs = 2)

simulating training for different learning rates... this may take a few moments...
Epoch 1/2
743/743 [==============================] - 22727s 31s/step - loss: 0.3050 - accuracy: 0.8932
Epoch 2/2
 21/743 [..............................] - ETA: 5:58:57 - loss: 0.3346 - accuracy: 0.9127

learner.fit_onecycle: This line initiates the training process of the model using a technique called "one-cycle learning." One-cycle learning is a strategy that adjusts the learning rate during training to potentially achieve faster convergence and better performance.


In [ ]:
learner.fit_onecycle(lr = 10e-3, epochs = 1)
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save('/content/drive/MyDrive/NLP/bertspam--classifier')